# FINAL TWITTER PORTION

In [ ]:
#Import Packages
import pandas as pd
import urllib
import time
import json
import sqlite3

# Number 1A

Create a 3rd table incorporating the Geo table (in addition to tweet and user tables that you already have) and extend your schema accordingly.
You will need to generate an ID for the Geo table primary key (you may use any value or combination of values as long as it is unique) for that table and link it to the Tweet table (foreign key should be in the Tweet table because there can be multiple tweets sent from the same location). In addition to the primary key column, the geo table should have “type”, “longitude” and “latitude” columns.


In [ ]:
#SQLite connection
conn = sqlite3.connect('tweets_DB.db')
cur = conn.cursor()

#Create tables
create_user_tbl = '''CREATE TABLE Users (
                id NUMBER(20),
                name VARCHAR(90),
                screen_name VARCHAR(90),
                description VARCHAR(90),
                friends_count INT
                );'''

create_geo_tbl = '''CREATE TABLE Geo ( 
                id NUMBER(20) PRIMARY KEY,
                Type VARYCHAR(50), 
                Longitude VARCHAR(50),
                Latitude VARCHAR(50)
                );'''

create_tweet_tbl = '''CREATE TABLE Tweets (
                id NUMBER(20) PRIMARY KEY, 
                Created_Date DATE, 
                Text CHAR(140),
                Source VARCHAR(200) DEFAULT NULL, 
                Reply_to_ID INT, 
                Reply_to_ScreenName VARCHAR(60), 
                Reply_to_StatusID INT, 
                Retweet_Cnt INT, 
                Contributors VARCHAR(200),
                user_id INT,
                geo_id INT,
                FOREIGN KEY (user_id) REFERENCES Users(id),
                FOREIGN KEY (geo_id) REFERENCES Geo(id)
                );'''

cur.execute(create_user_tbl)
cur.execute(create_geo_tbl)
cur.execute(create_tweet_tbl)

# Number 1B

Use python to download from the web and save to a local text file (not into database yet, just to text file) at least 500,000 lines worth of tweets. Test your code with fewer rows first – you can reduce the number of tweets if your computer is running too slow to handle 500K tweets. Report how long did it take

In [ ]:
#Test tweets with .txt file
wFD = urllib.request.urlopen('http://rasinsrv07.cstcis.cti.depaul.edu/CSC455/OneDayOfTweets.txt')
#Limit readlines to 500000
tweetLines = wFD.readlines(500000)
#Create .txt to write to
file = open('test_tweets.txt', 'w')

#Write tweets to file
for line in tweetLines:
    file.write(str(line))

#below is a process to see how long the file load took
start = time.time()
end = time.time()
print ("loadTweets took ", (end-start), ' seconds.')

file.close()

# Number 1C

Repeat what you did in part-b, but instead of saving tweets to the file, populate the 3-table schema that you created in SQLite. Be sure to execute commit and verify that the data has been successfully loaded (report row counts for each of the 3 tables).
How long did this step take?


# Load User Table

In [ ]:
#Users Load fucntion
def users_load(tweetLines):
    #Batch size
    batchRows = 50
    batchedInserts = []

#Insert loop
    while len(tweetLines) > 0:
        #Remove first line
        line = tweetLines.pop(0) 
        tweetDict = json.loads(line.decode('utf-8'))
        newRow = [] 
        #tags
        tweetKeys = ['id_str', 'name', 'screen_name', 'descriptions', 'friends_count']
        
        for key in tweetKeys:
            if key not in list(tweetDict.keys()):
                newRow.append(None)
            # Treat '', [] and 'null' as NULL
            elif tweetDict[key] in ['', [], 'null']:
                newRow.append(None)
            else:
                newRow.append(tweetDict[key])

        #Add the new row to the collected batch
        batchedInserts.append(newRow)

        #Check len
        if len(batchedInserts) >= batchRows or len(tweetLines) == 0:
            cur.executemany('INSERT INTO Users VALUES(?,?,?,?,?)', batchedInserts)
            # Reset the batching process
            batchedInserts = []

# Load Geo Table

In [ ]:
#Geo Load fucntion
def geo_load(tweetLines):
    #Batch size
    batchRows = 50
    batchedInserts = []

#Insert loop
    while len(tweetLines) > 0:
        #Remove first line
        line = tweetLines.pop(0) 
        tweetDict = json.loads(line.decode('utf-8'))
        newRow = [] 
        #tags
        tweetKeys = ['id_str', 'type', 'longitude', 'latitude']
        
        for key in tweetKeys:
            if key not in list(tweetDict.keys()):
                newRow.append(None)
            # Treat '', [] and 'null' as NULL
            elif tweetDict[key] in ['', [], 'null']:
                newRow.append(None)
            else:
                newRow.append(tweetDict[key])

        #Add the new row to the collected batch
        batchedInserts.append(newRow)

        #Check len
        if len(batchedInserts) >= batchRows or len(tweetLines) == 0:
            cur.executemany('INSERT INTO Geo VALUES(?,?,?,?)', batchedInserts)
            # Reset the batching process
            batchedInserts = []

# Load Tweet Table

In [ ]:
#Tweet Load fucntion
def tweet_load(tweetLines):
    #Batch size
    batchRows = 50
    batchedInserts = []

#Insert loop
    while len(tweetLines) > 0:
        #Remove first line
        line = tweetLines.pop(0) 
        tweetDict = json.loads(line.decode('utf-8'))
        newRow = [] 
        #tags
        tweetKeys = ['id_str','created_at','text','source','in_reply_to_user_id', 
        'in_reply_to_screen_name', 'in_reply_to_status_id', 'retweet_count', 'contributors', 'user_id', 'geo']
        
        for key in tweetKeys:
            if key not in list(tweetDict.keys()):
                newRow.append(None)
            # Treat '', [] and 'null as NULL
            elif tweetDict[key] in ['', [], 'null']:
                newRow.append(None)
            else:
                newRow.append(tweetDict[key])

        #Add the new row to the collected batch
        batchedInserts.append(newRow)

        #Check len
        if len(batchedInserts) >= batchRows or len(tweetLines) == 0:
            cur.executemany('INSERT INTO Tweets VALUES(?,?,?,?,?,?,?,?,?,?,?)', batchedInserts)
            # Reset the batching process
            batchedInserts = []

# Populate Database

NOTE: Doing a straight readlines() started an infinite loop so I had to truncate it

In [ ]:
#Connect to database
start = time.time()
conn = sqlite3.connect('tweets_DB.db')
cur = conn.cursor()
wFD = urllib.request.urlopen('http://rasinsrv07.cstcis.cti.depaul.edu/CSC455/OneDayOfTweets.txt')
tweetLines = wFD.readlines(4350000)

#Populate table
users_load(tweetLines)

#time
end = time.time()
print ("loadTweets took ", (end - start), ' seconds.')
print ("Loaded ", cur.execute('SELECT COUNT(*) FROM Users').fetchall()[0], " rows")
wFD.close()
conn.close()

In [ ]:
#Connect to database
start = time.time()
conn = sqlite3.connect('tweets_DB.db')
cur = conn.cursor()
wFD = urllib.request.urlopen('http://rasinsrv07.cstcis.cti.depaul.edu/CSC455/OneDayOfTweets.txt')
tweetLines = wFD.readlines(4350000)

#Populate table
geo_load(tweetLines)

#Time
end = time.time()
print ("loadTweets took ", (end - start), ' seconds.')
print ("Loaded ", cur.execute('SELECT COUNT(*) FROM Geo').fetchall()[0], " rows")
wFD.close()
conn.close()

In [ ]:
#Connect to database
start = time.time()
conn = sqlite3.connect('tweets_DB.db')
cur = conn.cursor()
wFD = urllib.request.urlopen('http://rasinsrv07.cstcis.cti.depaul.edu/CSC455/OneDayOfTweets.txt')
tweetLines = wFD.readlines(4350000)

#Populate table
tweet_load(tweetLines)

#Time
end = time.time()
print ("loadTweets took ", (end - start), ' seconds.')
print ("Loaded ", cur.execute('SELECT COUNT(*) FROM Tweets').fetchall()[0], " rows")
wFD.close()
conn.close()

## SQL Query 1

Find tweets where tweet id_str contains “44” or “77” anywhere in the column

In [ ]:
conn = sqlite3.connect('tweet_DB.db')
sql = '''SELECT * 
    FROM Tweets
    WHERE CONTAINS(id, '"44" OR "77')'''

pd.read_sql_query(sql, conn)

## SQL Query 2

Find how many unique values are there in the “in_reply_to_user_id” column

In [ ]:
conn = sqlite3.connect('tweet_DB.db')
sql = '''SELECT COUNT (DISTINCT in_reply_to_user_id) AS Count_Unique_ReplyIDs
    FROM Tweets'''

pd.read_sql_query(sql, conn)

## SQL Query 3

Find the average longitude and latitude value for each user name.

In [ ]:
conn = sqlite3.connect('tweet_DB.db')
sql = '''SELECT id, AVG(Longitude) AS Avg_Longitude, AVG(Latitude) AS Avg_Latitude
    FROM Geo
    GROUP BY id'''

pd.read_sql_query(sql, conn)